# Data Set

To train this CNN I have used a database with more than 31,500 images from 45 classes and have variety to wether, location and distance. 
<img src="ISatelitales/golf_course/golf_course_013.jpg" width=200 height=200>
<img src="ISatelitales/cloud/cloud_018.jpg" width=200 height=200> 
<img src="ISatelitales/ship/ship_011.jpg" width=200 height=200>

Publication of the database:  G. Cheng, J. Han, X. Lu. Remote Sensing Image Scene Classification: Benchmark and State of the Art. Proceedings of the IEEE.

### CODE

In [4]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random
import pandas as pd

### 1. Import images



In [6]:
dirname = os.path.join(os.getcwd(), 'ISatelitales')
imgpath = dirname + os.sep # os.sep es = \

imag = [] #list with images 
IDcat = [] #1 is could,0 is no cloud
IDim = [] #list of img index
idim = 0 #img index begin with 0



for ruta, carpetas, filenames in os.walk(imgpath): # path, folder and file names:

    print(ruta,idim)
    for filename in filenames: # looking at all files individually
            
        if ruta == imgpath +'cloud':  # if the image is in the cloud folder
            IDim.append(idim) ; idim += 1
            IDcat.append(1) # if the image have clouds IDcat value is 1
            filepath = os.path.join(ruta, filename) #obtain the image direction 
            imagen = plt.imread(filepath) #obtain the image array.
            imag.append(imagen) #save the image in images list
        else: 
            IDim.append(idim) ; idim += 1
            IDcat.append(0) # if the image have clouds IDcat value is 0
            filepath = os.path.join(ruta, filename) #obtain the image direction 
            imagen = plt.imread(filepath) #obtain the image array.
            imag.append(imagen) #save the image in images list
print('Total images: ',idim)
print('Cloud images: ',sum(IDcat))

F:\Cloud-detectionusing-CNN-Baby\ISatelitales\ 0
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\airplane 0
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\airport 700
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\baseball_diamond 1400
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\basketball_court 2100
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\beach 2800
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\bridge 3500
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\chaparral 4200
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\church 4900
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\circular_farmland 5600
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\cloud 6300
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\commercial_area 7000
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\dense_residential 7700
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\desert 8400
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\forest 9100
F:\Cloud-detectionusing-CNN-Baby\ISatelitales\freeway 9800
F:\Cloud-detectionusing-CNN-B

In [7]:
cat=IDcat
imagenes=imag

### 2. Division of the set into subsets training, validation and testing
It is necessary to separate the dataset into 3 groups:
* Train: will be used to train the CNN, it is the largest subset, it is 49% of the total images.
* Test: Used to calculate the error when the CNN is being trained, it is 21% of the total images.
* Validation: Used to validate the test score after training the CNN with a new data set, it is 30% of the total images.

This dataset has 31500 images, but only 700 have clouds, 2.2%. We have an imbalanced dataset, training a CNN with this ratio is a mistake. For this reason it is necessary to modify the ratio in the trining, test and validation dataset. After many tests, the best ratios are:
25% of images with clouds and 75% of images without clouds.

* Train: 30% of images with clouds.
* Test: 50% of images with clouds.
* Validation: 25% of images with clouds.



Cloud-free images filtering function

In [10]:
def filter_proportion(dat_set_cat, dat_set_im, rat_acept):
    Cat = [] 
    Im = []
    con1= 0 #counter Cloud-free images
    con2= 0 #counter cloud images
    lim = len(dat_set_cat)
    for i in range(lim) :
        if dat_set_cat[i] == 0:
            R=random.random()
            if R < rat_acept:
                con1 += 1
                Im.append(dat_set_im[i])
                Cat.append(dat_set_cat[i])

        elif dat_set_cat[i] == 1:
            con2 += 1
            Im.append(dat_set_im[i])
            Cat.append(dat_set_cat[i])


    Cat = np.array(Cat)
    Im = np.array(Im, dtype=np.uint8)
    print ('Ratio cloud images: ',con2/(con1+con2), '%')
    del dat_set_im; del dat_set_cat
    
    return Im, Cat
    

Separate the data set into data subsets

In [12]:
#subsets train-test and validation
Im,valIm1,Cat,valCat1 = train_test_split(imagenes,cat,test_size=0.3) 
del imagenes; del cat;

#subsets train test
entrIm1,testIm1,entrCat1,testCat1 = train_test_split(Im,Cat,test_size=0.3)
del Im; del Cat



In [1]:
#import keras
#from keras import optimizers
#from keras.utils import to_categorical

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from keras.models import Sequential,Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization
#from keras.layers.normalization import BatchNormalization
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU
from keras.models import load_model

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,  # 随机旋转的角度范围
    width_shift_range=0.2,  # 随机水平移动的范围
    height_shift_range=0.2,  # 随机垂直移动的范围
    shear_range=0.2,  # 剪切强度
    zoom_range=0.2,  # 随机缩放的范围
    horizontal_flip=True,  # 随机水平翻转
    fill_mode='nearest'
)

# 使用.flow_images()方法来增强图像

#Filtering unclouded images from subsets

In [ ]:
valIm, valCat = filter_proportion(valCat1, valIm1, rat_acept=0.07)

plt.figure(figsize=(20,10))
col = 5
for i in range(col):
    plt.subplot(5//col+1,col,i+1)
    plt.imshow(valIm[i])
    plt.title(valCat[i])

print(len(valIm))
print(len(valIm1))

In [ ]:
entrIm, entrCat = filter_proportion(entrCat1, entrIm1, rat_acept=0.05)

plt.figure(figsize=(20,10))
col = 5
for i in range(col):
    plt.subplot(5//col+1,col,i+1)
    plt.imshow(entrIm[i])
    plt.title(entrCat[i])

print(len(entrIm))
print(len(entrIm1))

In [ ]:
testIm, testCat = filter_proportion(testCat1, testIm1, rat_acept=0.025)

plt.figure(figsize=(20,10))
col = 5
for i in range(col):
    plt.subplot(5//col+1,col,i+1)
    plt.imshow(testIm[i])
    plt.title(testCat[i])

### 3. Image processing

All images are the same size, so the only step will be to normalize the pixel value in the arrays. This step is necessary for the best performance of the CNN.

In [ ]:
valIm = valIm / 255.
entrIm = entrIm / 255.
testIm = testIm / 255.

### 4. Develop the CNN

In [ ]:
from keras.layers import Layer
import keras.backend as K

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', 
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

Create the CNN structure

In [ ]:
modelo2 = Sequential()
modelo2.add(Conv2D(60, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(256,256,3)))
modelo2.add(MaxPooling2D((4, 4)))

modelo2.add(AttentionLayer()) 

modelo2.add(Conv2D(120, (3, 3), activation='relu')) #padding default is valid
modelo2.add(MaxPooling2D((2, 2)))

modelo2.add(Dropout(0.5))
modelo2.add(Conv2D(200, (3, 3), activation='relu'))
modelo2.add(MaxPooling2D((2, 2)))
modelo2.add(Conv2D(250, (3, 3), activation='relu'))

modelo2.add(Dropout(0.5))
modelo2.add(Flatten())
modelo2.add(Dense(512, activation='relu'))
modelo2.add(Dense(1,activation='sigmoid'))

In [ ]:
modelo2.summary()

### 5. Training the CNN

In [ ]:
import keras.backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC, PrecisionAtRecall

# 精确率
def precision(y_true, y_pred):
    # 确保y_pred的值在0和1之间
    y_pred = K.clip(y_pred, 0, 1)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# 召回率
def recall(y_true, y_pred):
    # 确保y_pred的值在0和1之间
    y_pred = K.clip(y_pred, 0, 1)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# F1-score
def f1_score(y_true, y_pred):
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# 特异性
def specificity(y_true, y_pred):
    y_pred = K.clip(y_pred, 0, 1)
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    specificity = true_negatives / (possible_negatives + K.epsilon())
    return specificity

In [ ]:
# 假设entrIm和entrCat是您的训练图像和标签
train_generator = datagen.flow(entrIm, entrCat, batch_size=32)

In [ ]:
modelo2.compile(optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['acc',precision, recall, f1_score, specificity])
#modelo2a = modelo2.fit(x=entrIm, y=entrCat, batch_size=32, epochs=30, verbose=1, validation_data=(testIm, testCat), shuffle=True)

In [ ]:
modelo2a=modelo2.fit(
    train_generator,
    steps_per_epoch=len(entrIm) / 32,  # 总样本数除以批量大小
    epochs=30,
    verbose=1,
    validation_data=(testIm, testCat),
    shuffle=True
)

save the model

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

# 在模型训练结束后，使用以下代码计算ROC-AUC和PR AUC
y_pred_proba = modelo2.predict(valIm)
roc_auc = roc_auc_score(valCat, y_pred_proba)
pr_auc = average_precision_score(valCat, y_pred_proba)

In [ ]:
#modelo2.save("modF6.h5py")

In [ ]:
3modelo2.save("modF6", save_format='tf')

### 6. CNN Evaluation
Test with validation Dataset

In [ ]:
test_eval = modelo2.evaluate(valIm, valCat, verbose=1)
 
print('Validation loss:', test_eval[0])
print('Validation accuracy:', test_eval[1])

Represent how the loss value and acc value change with epoch

In [ ]:
val_loss = modelo2a.history['val_loss'] #loss value
val_acc = modelo2a.history['val_acc'] #acc value
loss = modelo2a.history['loss'] #historical loss array
acc = modelo2a.history['acc'] #historical acc array

X1a1 = range(1, len(acc)+1)
plt.plot(X1a1, acc,'b', label='Training accurarcy')
plt.plot(X1a1, val_acc,'r', label='Test accurarcy')
plt.title('Training and Test accurarcy')
plt.legend()

plt.figure()
plt.plot(X1a1, loss, 'b', label='Training loss')
plt.plot(X1a1, val_loss, 'r',label='Test loss')
plt.title('Training and Test loss')
plt.legend()


plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 绘制精确率和召回率
plt.plot(historia.history['precision'], label='Precision')
plt.plot(historia.history['recall'], label='Recall')
plt.title('Precision and Recall')
plt.legend()

# 绘制F1-score和特异性
plt.plot(historia.history['f1_score'], label='F1-Score')
plt.plot(historia.history['specificity'], label='Specificity')
plt.title('F1-Score and Specificity')
plt.legend()

plt.show()

### 7. Validation and discussion

The values to loss and acc when the model is tested with the validation dataset was:
* The Test loss: 0.13231170177459717
* Test accuracy: 0.9465737342834473

This is a positive result, more than 94 images out of 100 were correctly classified.

In [ ]:
nub_pred = modelo2.predict(valIm, batch_size=32, verbose=1) 
nub_predicted = np.argmax(nub_pred, axis=1)  

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

Calculate the ROC curve values

In [ ]:
fpr, tpr, thresholds = roc_curve(valCat , nub_pred)

plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],'-')
plt.xlabel('Ratio (FPR)')
plt.ylabel('Sensibilidad (VPR)')
plt.title('Curve ROC')
plt.show()

The area of the curve

In [ ]:
AUC=roc_auc_score ( valCat ,  nub_pred) #Area curva roc
print("Area of curve ROC=",AUC)

Again, the result is so positive that the blue line of the ROC curve is close to 1 in the Y axis, which is the maximum possible value. The area of the curve is 0.9832888959371995, a very high and reliable value, the maximum possible value is 1.

Other evaluation values for the CNN. In this model the positive case is when it is an image with a cloud and the negative case is when it is an image without a cloud.

In [ ]:
dim = valCat.shape[0]
nv = 0; nf = 0; dv= 0; df = 0; na=0
cn = 0; cd= 0
inf=[]; idf=[]
for i in range(dim):
    if nub_pred[i] >= 0.5 and valCat[i] == 1:
        nv +=1
        cn +=1
    elif nub_pred[i] >= 0.5 and valCat[i] == 0:
        nf +=1
        inf.append(valIm[i])
        cd +=1
    elif nub_pred[i] < 0.5 and valCat[i] == 0:
        dv +=1
        cd +=1
    else :
        df +=1
        idf.append(valIm[i])
        cn +=1
    
print('True Positive TF=',nv)
print('False Positive FP=',nf)
print('True Negative TN=',dv)
print('False Negative FN=',df)
print('Sensitivity VPR=',nv/cn)
print('False positive rate FPR=',nf/cd)
print('Accuracy  ACC=',(nv+dv)/(cn+cd))
print('Specificity SPC=',dv/cd)
print('Positive Predictive Value PPV=',nv/(nv+nf))
print('Negative Predictive Value NPV=',dv/(dv+df))
print('Ratio FDR=',nf/(nf+nv))

#### The lowest values that this CNN has is the Sensitivity VPR= 0.94 versus the Specificity SPC= 0.979655717120500783, which means that this CNN is more efficient detecting images without clouds.

Some images that were not correctly predicted

In [ ]:
plt.figure(figsize=(20, 10))
col = 8
rows = (len(inf) // col) + (1 if len(inf) % col != 0 else 0)  # Calculate rows based on number of images
for i in range(len(inf)):
    plt.subplot(rows, col, i + 1)  # Update subplot grid size
    plt.imshow(inf[i])


In [ ]:
plt.figure(figsize=(20, 10))
col = 8
rows = (len(idf) // col) + (1 if len(idf) % col != 0 else 0)  # Calculate rows based on number of images
for i in range(len(idf)):
    plt.subplot(rows, col, i + 1)  # Update subplot grid size
    plt.imshow(idf[i])